# Asignación Financiera

## Librerias necesarias

In [2]:
import pandas as pd
import os
import pickle
import json
import xlsxwriter
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import re
from typing import Dict, List, Any
import itertools
from firebase_admin import credentials, firestore
import firebase_admin
import excel2img
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

## Datos del proyecto en analisis

In [3]:
proyecto="cabanaconde"
mes=4
anio=2025

## Datos de Firestore

In [4]:
# Ruta a tu archivo de clave privada descargado
cred = credentials.Certificate("../credentials_firestore.json")

# Inicializa sólo si no hay ninguna app por defecto
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)

# Obtiene una instancia de la base de datos Firestore
db = firestore.client()

In [5]:
data_mantenimiento=db.collection("rutinarios").document(proyecto).get()
if data_mantenimiento.exists:
    print(data_mantenimiento.to_dict())
else:
    print("No existe el documento.")

{'otros': {'tipo_superficie': 'afirmado', 'estado_conservacion_via': 'bueno'}, 'datos_generales': {'distritos': ['Cabanaconde'], 'region': 'Arequipa', 'provincia': 'Caylloma'}, 'expediente': {'codigo_ruta': 'R0405106', 'monto_total': 81120, 'coordenadas': {'fin': {'hemisferio': 'S', 'y': 8272525.68, 'progresiva': 13000, 'zona_letra': 'L', 'x': 813907.99, 'datum': 'WGS84', 'altitud': 1975, 'zona': 18}, 'inicio': {'hemisferio': 'S', 'y': 8272658.46, 'progresiva': 0, 'zona_letra': 'L', 'x': 819845.05, 'datum': 'WGS84', 'altitud': 2877, 'zona': 18}}, 'nombre': 'MANTENIMIENTO VIAL RUTINARIO DEL CAMINO VECINAL EMP. R0405104 (CABANACONDE) A PTE. CABANACONDE - CHOCO (L.P. CASTILLA), DISTRITO DE CABANACONDE, PROVINCIA DE CAYLLOMA, REGIÓN AREQUIPA', 'longitud': 13000}, 'other': {'tipo_superficie': 'afirmado', 'estado_conservacion_via': 'bueno'}, 'contrato': {'jefe_mantenimiento': {'titulo': 'Ingeniero', 'nombre': 'Genaro', 'dni': 0, 'apellido': 'Tinta Cáceres'}, 'numero_trabajadores': 4, 'denomi

In [6]:
progresiva_inicio=data_mantenimiento.to_dict()["expediente"]["coordenadas"]["inicio"]["progresiva"]
progresiva_fin=data_mantenimiento.to_dict()["expediente"]["coordenadas"]["fin"]["progresiva"]

print(progresiva_inicio)
print(progresiva_fin)

0
13000


In [7]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_vigente.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    df = pickle.load(f)

df.head(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
codigo,,,,,,,,,,
MR101,1.540444,2.310667,1.925556,2.310667,2.310667,2.310667,2.310667,1.925556,0.385111,17.33
MR102,153.148333,245.037333,214.407667,245.037333,245.037333,214.407667,245.037333,245.037333,30.629667,1837.78
MR103,0.000000,0.000000,6.650000,6.650000,6.650000,6.650000,0.000000,6.650000,0.000000,33.25
MR104,17.011250,17.011250,17.011250,17.011250,17.011250,17.011250,17.011250,17.011250,0.000000,136.09
MR201,1515.151818,2272.727727,1893.939773,2272.727727,2272.727727,1893.939773,2272.727727,1893.939773,378.787955,16666.67
MR202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR203,42.424545,42.424545,84.849091,84.849091,42.424545,42.424545,42.424545,42.424545,42.424545,466.67
MR204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [8]:
#eliminando el total
df = df.iloc[:-1]


In [9]:
df.index.to_list()

['MR101',
 'MR102',
 'MR103',
 'MR104',
 'MR201',
 'MR202',
 'MR203',
 'MR204',
 'MR205',
 'MR206',
 'MR301',
 'MR401',
 'MR501',
 'MR601',
 'MR701',
 'MR702']

In [10]:
ruta_actividades= os.path.join("data", "general_data", "actividades.json")
with open(ruta_actividades, 'r', encoding='utf-8') as archivo:
    actividades = json.load(archivo)
# Ahora 'datos' es un diccionario de Python
print(actividades)

[{'key': 'MR100', 'value': {'label': 'Conservación de calzada', 'value': [{'key': 'MR101', 'value': {'label': 'Limpieza de calzada', 'carga_trabajo': 3.6, 'unidad': 'Km'}}, {'key': 'MR102', 'value': {'label': 'Bacheo', 'carga_trabajo': 360, 'unidad': 'm2'}}, {'key': 'MR103', 'value': {'label': 'Desquinche', 'carga_trabajo': 0, 'unidad': 'm3'}}, {'key': 'MR104', 'value': {'label': 'Remoción de derrumbes', 'carga_trabajo': 9, 'unidad': 'm3'}}]}}, {'key': 'MR200', 'value': {'label': 'Limpieza de obras de arte', 'value': [{'key': 'MR201', 'value': {'label': 'Limpieza de cunetas', 'carga_trabajo': 1440, 'unidad': 'm'}}, {'key': 'MR202', 'value': {'label': 'Limpieza de alcantarillas', 'carga_trabajo': 0, 'unidad': 'unidad'}}, {'key': 'MR203', 'value': {'label': 'Limpieza de badén', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR204', 'value': {'label': 'Limpieza de zanjas de coronación', 'carga_trabajo': 0, 'unidad': 'm'}}, {'key': 'MR205', 'value': {'label': 'Limpieza de pontones', 'carga

In [11]:
ruta_cargas_trabajo= os.path.join("data", proyecto,str(mes), f"cargas_trabajo.json")
with open(ruta_cargas_trabajo, 'r', encoding='utf-8') as archivo:
    cargas_trabajo = json.load(archivo)
# Ahora 'cargas_trabajo' es un diccionario de Python
print(cargas_trabajo)

{'MR101': 3.6, 'MR102': 360, 'MR201': 1440, 'MR401': 10, 'MR104': 9}


In [12]:
{'MR101': 3.6, 'MR102': 360, 'MR201': 1440, 'MR401': 10, 'MR104': 9}

{'MR101': 3.6, 'MR102': 360, 'MR201': 1440, 'MR401': 10, 'MR104': 9}

In [13]:
[
    {"key": "MR101", "value": 3.6},
    {"key": "MR102", "value": 360},
    {"key": "MR104", "value": 9},
    {"key": "MR201", "value": 1440},
    {"key": "MR401", "value": 10},
]

[{'key': 'MR101', 'value': 3.6},
 {'key': 'MR102', 'value': 360},
 {'key': 'MR104', 'value': 9},
 {'key': 'MR201', 'value': 1440},
 {'key': 'MR401', 'value': 10}]

In [14]:
def ordenar_por_centena(data: Any) -> List[Dict[str, Any]]:
    """
    Toma un dict (o un JSON en formato str) cuyas claves acaban en número
    y devuelve una lista de dicts {'key':…, 'value':…} ordenada por ese número.
    """
    # Si viene como cadena JSON, lo convertimos
    if isinstance(data, str):
        data = json.loads(data)
    
    pattern = re.compile(r'(\d+)$')
    
    # Ordenamos items según el número de la clave
    sorted_items = sorted(
        data.items(),
        key=lambda item: int(pattern.search(item[0]).group(1))
    )
    
    # Devolvemos la lista esperada
    return [{"key": k, "value": v} for k, v in sorted_items]

# Ejemplo completo
ruta_cargas_trabajo = '{"MR101": 3.6, "MR102": 360, "MR201": 1440, "MR401": 10, "MR104": 9}'
resultado = ordenar_por_centena(ruta_cargas_trabajo)
print(resultado)
# → [
#     {'key': 'MR101', 'value': 3.6},
#     {'key': 'MR102', 'value': 360},
#     {'key': 'MR104', 'value': 9},
#     {'key': 'MR201', 'value': 1440},
#     {'key': 'MR401', 'value': 10},
# ]

[{'key': 'MR101', 'value': 3.6}, {'key': 'MR102', 'value': 360}, {'key': 'MR104', 'value': 9}, {'key': 'MR201', 'value': 1440}, {'key': 'MR401', 'value': 10}]


In [15]:
def clave_grupo(item):
    return int(item['key'][2:]) // 100  # 101→1, 201→2, etc.

In [16]:
def calcular_avance(programado, ejecutado):
    """
    Calcula el porcentaje de avance basado en lo programado y lo ejecutado.
    
    Si lo programado es 0 y lo ejecutado es mayor a 0, devuelve 'ejecución adelantada'.
    Si ambos son 0, devuelve 0.0.
    En cualquier otro caso, devuelve el porcentaje (0-100) como float redondeado a 2 decimales.
    """
    if programado == 0:
        if ejecutado > 0:
            return "Ejec. adelantada"
        else:
            return 0.0
    else:
        porcentaje = (ejecutado / programado) 
        return round(porcentaje, 2)


In [17]:
def formatear_progresiva(distancia, decimales=0):
    """
    Convierte una distancia en metros a notación de progresiva.
    
    Parámetros:
    - distancia: int o float, la distancia en metros.
    - decimales: int, número de decimales a mostrar en la parte de los metros.
    
    Retorna:
    - str: progresiva en formato 'K+XXX' con los decimales indicados.
    """
    if not isinstance(distancia, (int, float)):
        raise ValueError("La distancia debe ser un número (int o float).")
    if not isinstance(decimales, int) or decimales < 0:
        raise ValueError("Los decimales deben ser un entero no negativo.")
    
    km = int(distancia) // 1000
    metros = distancia - (km * 1000)
    
    formato_metros = f"{metros:0.{decimales}f}".zfill(3 + (1 if decimales > 0 else 0) + decimales)
    return f"{km}+{formato_metros}"

In [18]:
cargas_trabajo_ordenadas=ordenar_por_centena(ruta_cargas_trabajo)
print(cargas_trabajo_ordenadas)

[{'key': 'MR101', 'value': 3.6}, {'key': 'MR102', 'value': 360}, {'key': 'MR104', 'value': 9}, {'key': 'MR201', 'value': 1440}, {'key': 'MR401', 'value': 10}]


## Escribiendo el excel

In [19]:
# escribiendo una tabla de excel

In [20]:
# Ruta dinámica
ruta_directorio = os.path.join("output", proyecto, str(mes))
ruta_archivo = os.path.join(ruta_directorio, "cargas_trabajo.xlsx")
# Crear carpetas si no existen
os.makedirs(ruta_directorio, exist_ok=True)

wb = xlsxwriter.Workbook(ruta_archivo)
ws=wb.add_worksheet("cargas_trabajo")

### Formatos

In [21]:
# 1. Propiedades base
BASE_CELL = {
    "valign": "vcenter",
    "border": 1,
}

BASE_HEADER = {
    **BASE_CELL,
    "bold": True,
}

BG_HEADER = "#D9E1F2"
BG_TABLE = "#C5D9F1"


# 2. Función de helper para crear formatos
def fmt(wb, **props):
    cfg = {}
    # parte genérica
    cfg.update(props)
    return wb.add_format(cfg)


# 3. Diccionario de formatos
formats = {
    # encabezados
    "header": {"align": "center", "bg_color": BG_HEADER, **BASE_HEADER},
    "header2": {"align": "left", **BASE_HEADER},
    "header_label": {"align": "center", **BASE_HEADER},
    # formato vertical header
    "header_vertical": {
        **BASE_HEADER,
        "bold": True,
        "align": "center",
        "valign": "vcenter",
        "bg_color": BG_HEADER,
        "rotation": 90,  # <— ¡aquí está el giro!
    },
    # encabezados de tabla
    "table_header": {
        "align": "center",
        "text_wrap": True,
        "shrink": True,
        "bg_color": BG_TABLE,
        **BASE_CELL,
    },
    "table_header_int": {"align": "center", "bg_color": BG_TABLE, **BASE_CELL},
    # celdas numéricas y de texto
    "cell": {"font_size": 8, "num_format": "#,##0.00", **BASE_CELL},
    # "cell": {"font_size": 8, "num_format": "#,##0.00", **BASE_CELL},
    "cell_int": {"num_format": "#,##0", **BASE_CELL},
    "cell_number": {"num_format": "#,##0.00", **BASE_CELL},
    "cell_number_percent": {
        "num_format": "#,##0.00%",
        "align": "right",
        **BASE_CELL,
    },
    "cell_text": {"align": "left", "text_wrap": True, "shrink": True, **BASE_CELL},
    "cell_right": {"align": "right", **BASE_CELL},
    # sumas
    "suma": {"num_format": "#,##0.00", **BASE_CELL},
    # soles
    "soles_color": {
        "bold": True,
        "bg_color": BG_HEADER,
        "num_format": '"S/." #,##0.00',
        **BASE_CELL,
    },
    "soles": {"num_format": '"S/." #,##0.00', **BASE_CELL},
    # formato de ajuste general
    "ajustar": {"align": "center", "text_wrap": True, "shrink": True, **BASE_CELL},
    "only_border": {
        "border": 1,
    },
}

# 4. Creación dinámica de los objetos Format
wb_formats = {name: fmt(wb, **props) for name, props in formats.items()}

# Ahora sólo usa wb_formats["header"], wb_formats["cell_text"], etc.

In [22]:
# Punto de inserción (cero-indexed)
table_start_row = 6   # si quieres que empiece en la fila 7
table_start_col = 1   # si quieres que empiece en la columna B

In [23]:
def merge(ws, r1, c1, r2, c2, valor, fmt):
    """Merge relative a table_start_* + offsets."""
    ws.merge_range(
        xl_range(table_start_row + r1,
                 table_start_col + c1,
                 table_start_row + r2,
                 table_start_col + c2),
        valor, fmt
    )

In [24]:
def write_rel(ws, row_offset, col_offset, value, fmt=None):
    """
    Escribe en la celda (table_start_row + row_offset, table_start_col + col_offset)
    """
    abs_row = table_start_row + row_offset
    abs_col = table_start_col + col_offset
    ws.write(abs_row, abs_col, value, fmt)


In [25]:
# 2. Helpers con tuplas (x, y)

def merge_xy(ws, start: tuple[int,int], end: tuple[int,int], value, fmt):
    """
    Merge de celdas desde start=(x1, y1) hasta end=(x2, y2),
    relativas a table_start_col/row.
    """
    (x1, y1), (x2, y2) = start, end
    ws.merge_range(
        xl_range(
            table_start_row + y1,  # row1 absoluto
            table_start_col + x1,  # col1 absoluto
            table_start_row + y2,  # row2 absoluto
            table_start_col + x2,  # col2 absoluto
        ),
        value, fmt
    )

def write_xy(ws, pos: tuple[int,int], value, fmt=None):
    """
    Escribe un valor en pos=(x, y) relativa a table_start_col/row.
    """
    x, y = pos
    ws.write(
        table_start_row + y,
        table_start_col + x,
        value, fmt
    )

In [26]:
def set_columns_rel(ws, rel_col_widths):
    """
    Aplica ws.set_column a rangos de columnas definidos
    en coordenadas relativas (x) a table_start_col.
    rel_col_widths: lista de tuplas (first_rel, last_rel, width)
    """
    for first_rel, last_rel, width in rel_col_widths:
        abs_first = table_start_col + first_rel
        abs_last  = table_start_col + last_rel
        ws.set_column(abs_first, abs_last, width)

In [27]:
col_widths_rel = [
    (0, 0, 8),  # columna B en adelante:  B–B ancho=8
    (1, 1, 12),  # C–C ancho=30
    (2, 2, 30),  # D–H ancho=8
    (3, 3, 12),  # I–T ancho=8
    (4, 4, 10),  # U–U ancho=10
    (5, 5, 10),  # U–U ancho=10
    (6, 6, 14),  # U–U ancho=10
    (7, 7, 14),
]

set_columns_rel(ws, col_widths_rel)

In [28]:
merge_xy(ws, (0, 0), (0, 1), "N.º", wb_formats["header"])
merge_xy(ws, (1, 0), (1, 1), "Código", wb_formats["header"])
merge_xy(ws, (2, 0), (2, 1), "Actividades", wb_formats["header"])

merge_xy(ws, (3, 0), (3, 1), "Unidad", wb_formats["header"])

merge_xy(ws, (4, 0), (5, 0), "Cargas de trabajo", wb_formats["header"])

write_xy(ws, (4, 1), "Programacion", wb_formats["header_vertical"])
write_xy(ws, (5, 1), "Ejecución", wb_formats["header_vertical"])

merge_xy(ws, (6, 0), (6, 1), "Avance", wb_formats["header"])
merge_xy(ws, (7, 0), (7, 1), "Progresivas", wb_formats["header"])


In [29]:
current_col = 0
current_row = 2

In [30]:
print(current_row, current_col)

2 0


In [31]:
codigo = "MR102"
mes_int = 4
anio = 2025

# Formatea el mes con dos dígitos y crea "2025-04"
col = f"{anio}-{mes_int:02d}"

valor = df.loc[codigo, col]
print(type(valor))  # 153.14833…

<class 'numpy.float64'>


In [32]:
columnas_insertadas = 0

impresos = set()

contador_items = 1

for grupo, elementos in itertools.groupby(cargas_trabajo_ordenadas, clave_grupo):
    current_col = 0

    # codigo=int(carga_trabajo['key'][2:])
    # grupo=codigo//100
    # print(grupo)

    inicio = grupo * 100
    fin = inicio + 99

    print(f"Valores entre {inicio} al {fin}!")
    resultado = next(
        (item for item in actividades if item["key"] == f"MR{inicio}"), None
    )
    # print(resultado)

    write_xy(ws, (current_col, current_row), "", wb_formats["cell_text"])  # numero
    current_col += 1
    write_xy(
        ws, (current_col, current_row), resultado["key"], wb_formats["cell_text"]
    )  # codigo
    current_col += 1
    write_xy(
        ws,
        (current_col, current_row),
        resultado["value"]["label"],
        wb_formats["cell_text"],
    )  #

    current_row += 1

    for it in elementos:
        current_col = 0
        print(f"  {it['key']}: {it['value']}")

        resultado2 = next(
            (item for item in resultado["value"]["value"] if item["key"] == it["key"]),
            None,
        )
        print(resultado)

        write_xy(
            ws, (current_col, current_row), contador_items, wb_formats["cell_int"]
        )  # numero
        current_col += 1
        write_xy(
            ws, (current_col, current_row), resultado2["key"], wb_formats["cell_text"]
        )
        current_col += 1
        write_xy(
            ws,
            (current_col, current_row),
            resultado2["value"]["label"],
            wb_formats["cell_text"],
        )
        current_col += 1
        write_xy(
            ws,
            (current_col, current_row),
            resultado2["value"]["unidad"],
            wb_formats["cell_text"],
        )
        current_col += 1
        write_xy(
            ws,
            (current_col, current_row),
            df.loc[it["key"], f"{anio}-{mes:02d}"],
            wb_formats["cell_number"],
        )
        current_col += 1
        write_xy(
            ws,
            (current_col, current_row),
            cargas_trabajo[it["key"]],
            wb_formats["cell_number"],
        )
        current_col += 1

        avance = calcular_avance(
            programado=df.loc[it["key"], f"{anio}-{mes:02d}"],
            ejecutado=cargas_trabajo[it["key"]],
        )
        write_xy(
            ws,
            (current_col, current_row),
            avance,
            wb_formats["cell_number_percent"],
        )
        current_col += 1

        progresivas = (
            formatear_progresiva(progresiva_inicio)
            + " - "
            + formatear_progresiva(progresiva_fin)
        )
        write_xy(
            ws,
            (current_col, current_row),
            progresivas,
            wb_formats["cell_number_percent"],
        )
        current_col += 1

        #######3
        contador_items += 1
        current_row += 1

Valores entre 100 al 199!
  MR101: 3.6
{'key': 'MR100', 'value': {'label': 'Conservación de calzada', 'value': [{'key': 'MR101', 'value': {'label': 'Limpieza de calzada', 'carga_trabajo': 3.6, 'unidad': 'Km'}}, {'key': 'MR102', 'value': {'label': 'Bacheo', 'carga_trabajo': 360, 'unidad': 'm2'}}, {'key': 'MR103', 'value': {'label': 'Desquinche', 'carga_trabajo': 0, 'unidad': 'm3'}}, {'key': 'MR104', 'value': {'label': 'Remoción de derrumbes', 'carga_trabajo': 9, 'unidad': 'm3'}}]}}
  MR102: 360
{'key': 'MR100', 'value': {'label': 'Conservación de calzada', 'value': [{'key': 'MR101', 'value': {'label': 'Limpieza de calzada', 'carga_trabajo': 3.6, 'unidad': 'Km'}}, {'key': 'MR102', 'value': {'label': 'Bacheo', 'carga_trabajo': 360, 'unidad': 'm2'}}, {'key': 'MR103', 'value': {'label': 'Desquinche', 'carga_trabajo': 0, 'unidad': 'm3'}}, {'key': 'MR104', 'value': {'label': 'Remoción de derrumbes', 'carga_trabajo': 9, 'unidad': 'm3'}}]}}
  MR104: 9
{'key': 'MR100', 'value': {'label': 'Conser

In [33]:
print(current_row, current_col)

10 8


In [34]:
print(
    xl_range(
        table_start_row,
        table_start_col,
        table_start_row + current_row-1,
        table_start_col+current_col-1,
    ),
)

B7:I16


In [35]:
# colocar grilla  a toda la tabla
ws.conditional_format(
    xl_range(
        table_start_row,
        table_start_col,
        table_start_row + current_row - 1,
        table_start_col + current_col - 1,
    ),
    {"type": "no_errors", "format": wb_formats["only_border"]},
)

0

In [36]:
# Guardar archivo
wb.close()

In [37]:
# 1. Carga el libro y la hoja

wb = load_workbook(ruta_archivo, data_only=True)
ws = wb.active  # o wb["NombreHoja"]

# 2. Determina el rango usado
min_row, min_col = ws.min_row, ws.min_column
max_row, max_col = ws.max_row, ws.max_column
rango = f"{get_column_letter(min_col)}{min_row}:{get_column_letter(max_col)}{max_row}"

# 3. Exporta a PNG
salida_png = f"cargas_trabajo.png"
ruta_salida_image= os.path.join(ruta_directorio, salida_png)
excel2img.export_img(ruta_archivo, ruta_salida_image, ws.title, rango)

print(f"Imagen generada: {salida_png}")


Imagen generada: cargas_trabajo.png
